In [184]:
import os
from os.path import isfile, join, dirname, abspath
import sys
import re
import argparse
import shutil
import subprocess
import tempfile
import pandas as pd

In [209]:
## read the cavity synthesis
synthese_cavites = pd.read_csv("../data/cadastre/UP_MDD_DDA_Temp_synthese_Cavites.csv")

In [210]:
synthese_cavites.index = synthese_cavites['NomCadastre']

In [215]:
for dirpath, dirnames, filenames in os.walk('../data'):
    # check along each directory.
    for name in dirnames:
        # find all cave folders, each has a three digit cadastre sector identifier in path
        if not re.match('\d{3}', name):
            if re.match('.*\d{3}',dirpath) and (name !='outputs'):
                #print('yay!', name,dirpath)
                try:
                    print(synthese_cavites.loc[name]['NomCadastre'])
                except KeyError:
                    print(name)
                    print("cette grotte n'est pas dans le xls")

MD-BDG05
GouffreAguilucho
PorcheDeLaMadre2
AbrigoDelViento
MD-BDGS04
MD-BDGS03
GrottePetiteSoeur
PuitsOctopus
MD-BDGS02
IgueDeDob
NomCadastre
GouffreDeLArche    GouffreDeLArche
GouffreDeLArche    GouffreDeLArche
Name: NomCadastre, dtype: object
GrandAvenTriple
GrotteDuNomade
GrotteGrandeSoeur
GrotteMarion
AvenDeLaTraverse
GrotteDeLaConduite
GrotteFilostik
GouffreDesMousquetaires
Gouffre36
PorcheDeLImpatience
MeandreDesSables
MD-BDGS01
PerteDesLobos
Perte1Lobos
ExsurgencePicaflor
LaGrotte
GouffreCumpleAnos
ElCuloDeLaMadre
CuevaDelAgua
LobosMatos
SumideroPiquante
GrotteDeLobosSecos
SimaDelCol
ResurgenceFinDeSeno
GrotteFaussePucelle
GrotteCacahuete
GrotteDuRapace
GouffreAlouette
GrotteDeLOursin
EvequeCauchon
GouffreGazelle
TrouDesCanelures
AbriDuGazon
PuitsDuBourdon
GrotteDuNord
PerteDuCoypo
GrosTrou
GouffreFelin
PerteFaille
JamesBonde
GrotteSueno
IgueDuPlouc
AbriDuCerfeuil
CuevaArdilla
PerteDuJump
PerteInattendue
SumideroJuanito
PerteAmbree
Sou33
FailleDuCoinceur
CuevaLaBas
MD-SOU02
Gold

In [ ]:
re.match?